In [57]:
import pandas as pd
import numpy as np
import re
from gensim import models
from time import time
from textblob import TextBlob

### Reading CSV

In [58]:
df = pd.read_csv('youtube_chat.csv', engine = 'python')
#remove rows with missing comments
df.dropna(subset = ["Comment"], inplace=True)

### Cleaning Data

In [59]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def remove_punctuation(text):

    final = re.sub("@[A-Za-z0-9_]+","", text)#remove names from tweets
    final = re.sub(r"http\S+", "", final)#remove url from tweets
    final = "".join(u for u in final if u not in ("?",".","_", ";", ":", "," , "!",'"'))#remove punctuation
    final = remove_emoji(final)

    return final

In [78]:
df['Comments'] = df['Comment'].apply(remove_punctuation)

### Adding Polarity

In [74]:
df['polarity'] = df['Comments'].apply( lambda x: TextBlob(x).polarity)
df['Sentiment']  = np.where(df['polarity'] > 0, 1,-1)

### Splitting Data

In [79]:
dfNew = df[['Comments','Sentiment']]
# random split train and test data
index = df.index
df['random_number'] = np.random.randn(len(index))
train = df[df['random_number'] <= 0.8]
test = df[df['random_number'] > 0.8]




### Training ML classifiers

In [92]:

# count vectorizer:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Comments'])
test_matrix = vectorizer.transform(test['Comments'])



# Logistic Regression and Neural Network
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
#nn = MLPClassifier()
lr = LogisticRegression()

In [93]:
X_train = train_matrix
X_test = test_matrix
y_train = train['Sentiment']
y_test = test['Sentiment']

In [86]:
#train logistic Regression
#lr.fit(X_train,y_train)
nn.fit(X_train,y_train)

MLPClassifier()

In [88]:
predictions = nn.predict(X_test)
predictions

array([-1, -1, -1, ...,  1,  1,  1])

### Performance measures

In [94]:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

array([[3907,   12],
       [   2,  247]])

In [95]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00      3919
           1       0.95      0.99      0.97       249

    accuracy                           1.00      4168
   macro avg       0.98      0.99      0.99      4168
weighted avg       1.00      1.00      1.00      4168

